### Import packages

In [2]:
import os
import glob
import zipfile
import shutil
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
import math
import codecs
import nltk

### Set all important file paths

In [1]:
pwd()

u'/Users/Richard/Desktop/Berkeley/w266/repo/w266_project'

In [3]:
# At work:
TEXT_PATH = "T:/Quant/TextAnalysis/Transcripts/SP100/Text/"
PDF_PATH = "T:/Quant/TextAnalysis/Transcripts/SP100/PDF/"
LIBRARY_PATH = "T:/Quant/TextAnalysis/Transcripts/SP100/Libraries/"

# At home:
dict_path = '/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/'
input_path = '/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/'
output_path = '/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/'



### Create some test documents

In [4]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

test_string = ('Okay? Thanks, Don! Next-slide is a 493 little bit of detail on the significant charges, with the Pizza'
               ' Sprint and German acquisitions. As you can see, we\'ve got the split now between Pizza Sprint'
               ' and Germany.')
test_string2 = 'If not staying at the XXXX please note that East Malvern footy club is right at Darling station (see' \
               ' train line map below). To make it easy for yourself to get around by train it is recommended that you' \
               ' stay somewhere close to a train station that is on the Glen Waverley train line' \
               ' (the full train line is shown below). From there you will be able to easily get to and from,' \
               ' East Malvern Knights Footy Club (Darling station), Etihad (Southern Cross station) and the MCG' \
               ' (Richmond or Parliament stations).'
test_string3 = "It's your toy, not Richard's."
test_strings = [test_string, test_string2, test_string3]


### Import a dictionary

Import a bag of words style dictionary for word counting analyses


In [14]:
def get_dictionary(file_path, file_name):
    """Read a standard word list dictionary text file into a list"""
    with open(file_path + file_name, "r") as file:
        words = [word.lower().rstrip('\n') for word in file]
    words = [word.rstrip('\r') for word in words]
    return words


#### Example

In [15]:
function_words = get_dictionary(dict_path,'function_words.txt')

In [16]:
print (function_words[:10])

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost']


### Function to collect file paths

In [17]:
# Get the file paths to all files of a specified file type from a given directory
def get_files(file_path, file_type):
    file_list = glob.glob(file_path + "*." + file_type)
    return file_list


#### Example

In [93]:
# Get all the file paths into a list
TextFiles = []
for path in ['T1/','T2/','T3/','T4/']:
    TextFiles += get_files(input_path + path,'txt')
# PDFFiles = get_files(PDF_PATH,'pdf')


In [20]:
TextFiles[:5]

['/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/T1/20100730_1_AEP.txt',
 '/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/T1/20100730_1_CVX.txt',
 '/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/T1/20100730_1_MET.txt',
 '/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/T1/20100730_1_MRK.txt',
 '/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/T1/20100730_1_SPG.txt']

In [94]:
len(TextFiles)

3248

### Check files have the correct identifier


In [23]:
def quick_read_transcript(file_path):
    # read in all lines of the transcript
    file_code = file_path.split('_')[-1][:-4]
    
    i = 0
    results = []

    with open(file_path, "r") as file:
        while i < 4:
            try:
                results.append(file.readline().rstrip('\n').rstrip('\r')) # remove new line and any surrounding spaces
                i += 1
            except UnicodeDecodeError:
                results = ["UnicodeDecodeError in file"]
                i = 4
    try:
        code = results[1].split(': ')[1].split(' ')[0]
    except:
        code = result[1]
    try:
        event = results[3].split(': ')[1]
    except:
        event = results[3]
    return (file_code, code, event)
    
quick_read_transcript(TextFiles[0])   

('AEP', 'AEP', 'Q2 2010 Earnings Call')

In [25]:
i = 0
for tf in sorted(TextFiles):
    result = quick_read_transcript(tf)
    if result[0] != result[1]:
        print (tf.split('w266_project/')[1], result)
    i += 1
    

('T1/20101027_1_0848680D.txt', ('0848680D', 'S', 'Q3 2010 Earnings Call'))
('T1/20110428_1_0848680D.txt', ('0848680D', 'S', 'Q1 2011 Earnings Call'))
('T1/20110714_1_GOOGL.txt', ('GOOGL', 'GOOG', 'Q2 2011 Earnings Call'))
('T1/20110728_1_0848680D.txt', ('0848680D', 'S', 'Q2 2011 Earnings Call'))
('T1/20111013_1_GOOGL.txt', ('GOOGL', 'GOOG', 'Q3 2011 Earnings Call'))
('T1/20111026_1_0848680D.txt', ('0848680D', 'S', 'Q3 2011 Earnings Call'))
('T1/20120119_1_GOOGL.txt', ('GOOGL', 'GOOG', 'Q4 2011 Earnings Call'))
('T1/20120208_1_0848680D.txt', ('0848680D', 'S', 'Q4 2011 Earnings Call'))
('T1/20120412_1_GOOGL.txt', ('GOOGL', 'GOOG', 'Q1 2012 Earnings Call'))
('T1/20120425_1_0848680D.txt', ('0848680D', 'S', 'Q1 2012 Earnings Call'))
('T1/20120719_1_GOOGL.txt', ('GOOGL', 'GOOG', 'Q2 2012 Earnings Call'))
('T1/20120726_1_0848680D.txt', ('0848680D', 'S', 'Q2 2012 Earnings Call'))
('T1/20121018_1_GOOGL.txt', ('GOOGL', 'GOOG', 'Q3 2012 Earnings Call'))
('T1/20121025_1_0848680D.txt', ('0848680D',

Two stocks have the wrong code in their headers, but they are the correct stocks

### Check which are not Earnings Call's

In [26]:
i = 0
for tf in sorted(TextFiles):
    result = quick_read_transcript(tf)
    if result[2].find('Earnings Call') == -1:
        print (tf.split('w266_project/')[1], result)
    i += 1


('T1/20101019_1_AEP.txt', ('AEP', 'AEP', 'Analyst Day'))


Just one is not an Earnings Call, so remove

### Determine if we are missing any calls

In [27]:
# Create a set of all possible calls in our lookback window
calls_full_set = []
for year in range(2010,2019):
    for quarter in range(1,5):
        calls_full_set.append(str(year) + '_' + str(quarter))
print (calls_full_set)


['2010_1', '2010_2', '2010_3', '2010_4', '2011_1', '2011_2', '2011_3', '2011_4', '2012_1', '2012_2', '2012_3', '2012_4', '2013_1', '2013_2', '2013_3', '2013_4', '2014_1', '2014_2', '2014_3', '2014_4', '2015_1', '2015_2', '2015_3', '2015_4', '2016_1', '2016_2', '2016_3', '2016_4', '2017_1', '2017_2', '2017_3', '2017_4', '2018_1', '2018_2', '2018_3', '2018_4']


In [28]:
# Script to reorganise event description
def tidy_call_text(text):
    text = text.split(' ')
    call = text[1] + '_' + text[0][-1:]
    return call

# Script to see if there are any events missing between the first and last recorded events 
def find_missing(stock_list, full_list):
    first_q = stock_list[0]
    last_q = stock_list[-1]
    start = full_list.index(first_q)
    end = full_list.index(last_q) + 1
    adj_full_list = full_list[start:end]
    missing = []
    for q in adj_full_list:
        if q not in stock_list:
            missing.append(q)
    return missing

# Script to see if there are any events that are duplicates 
def duplicate_calls(call_list):
    duplicates = []
    counts = {call: call_list.count(call) for call in call_list}
    duplicates = [key for key, value in counts.items() if value > 1]
    return duplicates
    

In [29]:
 
all_calls ={}

for tf in sorted(TextFiles):
    result = quick_read_transcript(tf)
    if result[2].find('Earnings Call') != -1 and result[2].find('Fixed') == -1:   # Found an earnings call
        ticker = result[1]
        tidy_call = tidy_call_text(result[2])
        if result[1] not in all_calls.keys():
            all_calls[ticker] = [tidy_call]
        else:
            all_calls[ticker].append(tidy_call)

for ticker in sorted(all_calls.keys()):
    print (ticker, sorted(all_calls[ticker]))
            


('AA', ['2016_4', '2017_1', '2017_2'])
('AAPL', ['2010_4', '2011_1', '2011_3', '2011_4', '2012_1', '2012_2', '2012_3', '2012_4', '2013_1', '2013_2', '2013_3', '2013_4', '2014_1', '2014_2', '2014_3', '2014_4', '2015_1', '2015_2', '2015_3', '2015_4', '2016_1', '2016_2', '2016_3', '2016_4', '2017_1', '2017_2', '2017_3'])
('ABBV', ['2013_1', '2013_2', '2013_3', '2013_4', '2014_1', '2014_2', '2014_3', '2014_4', '2015_1', '2015_2', '2015_3', '2015_4', '2016_1', '2016_2', '2016_3', '2016_4', '2017_1', '2017_2'])
('ABT', ['2010_3', '2010_4', '2011_2', '2011_3', '2011_4', '2012_1', '2012_2', '2012_3', '2012_4', '2013_1', '2013_2', '2013_3', '2013_4', '2014_1', '2014_2', '2014_3', '2014_4', '2015_1', '2015_2', '2015_3', '2015_4', '2016_1', '2016_2', '2016_3', '2016_4', '2017_1', '2017_2'])
('ACN', ['2010_4', '2011_1', '2011_3', '2011_4', '2012_1', '2012_2', '2012_3', '2012_4', '2013_1', '2013_2', '2013_3', '2013_4', '2014_1', '2014_2', '2014_3', '2014_4', '2015_1', '2015_2', '2015_3', '2015_4', 

### Missing calls?

In [30]:
for ticker in sorted(all_calls.keys()):
    missing = find_missing(all_calls[ticker],calls_full_set)
    if len(missing) > 0:
        #print (ticker, 'includes:', all_calls[ticker])
        print (ticker, 'missing:', missing)
        print ()

('AAPL', 'missing:', ['2011_2'])
()
('ABT', 'missing:', ['2011_1'])
()
('ACN', 'missing:', ['2011_2'])
()
('AEP', 'missing:', ['2010_3', '2010_4', '2011_4', '2012_4', '2016_3'])
()
('AGN', 'missing:', ['2010_4', '2013_4', '2014_4'])
()
('AIG', 'missing:', ['2010_4'])
()
('ALL', 'missing:', ['2010_4'])
()
('AMGN', 'missing:', ['2011_1'])
()
('AMZN', 'missing:', ['2010_4'])
()
('AVP', 'missing:', ['2010_4'])
()
('AXP', 'missing:', ['2011_1'])
()
('BAC', 'missing:', ['2011_1'])
()
('BAX', 'missing:', ['2010_4'])
()
('BHI', 'missing:', ['2015_1', '2015_2', '2015_3', '2015_4', '2016_1'])
()
('BIIB', 'missing:', ['2010_4'])
()
('BK', 'missing:', ['2011_1'])
()
('BMY', 'missing:', ['2010_4'])
()
('C', 'missing:', ['2011_1'])
()
('CAT', 'missing:', ['2010_4'])
()
('CELG', 'missing:', ['2010_4'])
()
('CHTR', 'missing:', ['2010_4'])
()
('CL', 'missing:', ['2010_4'])
()
('CMCSA', 'missing:', ['2010_4'])
()
('COF', 'missing:', ['2011_2'])
()
('COST', 'missing:', ['2011_2'])
()
('CPB', 'missing:', 

NOte:
    
- **GOOGL** are not missing just have an incorrect code in the text file

- **LOW** are not missing just have a year typo in the file

- **F** have 'Fixed' in their event description

Otherwise about 30 odd missing ore around 1%.



### Duplicate calls?

In [31]:
dup_2012_4 = []

for ticker in sorted(all_calls.keys()):
    duplicates = duplicate_calls(all_calls[ticker])
    if len(duplicates) > 0:
        if len([i for i in duplicates if i != '2012_4']) > 0:
            print (ticker, 'duplicates:', [i for i in duplicates if i != '2012_4'])
        if '2012_4' in duplicates:
            dup_2012_4.append(ticker)

print ()
print ('2012_4', dup_2012_4)


('HNZ', 'duplicates:', ['2014_2', '2014_1'])
('LOW', 'duplicates:', ['2016_1', '2016_2', '2016_4'])
('UTX', 'duplicates:', ['2014_4'])
('WMB', 'duplicates:', ['2016_4'])
()
('2012_4', ['AIG', 'AMZN', 'BAC', 'C', 'CAT', 'CMCSA', 'DIS', 'DOW', 'DVN', 'FB', 'FOXA', 'GE', 'GS', 'HAL', 'HPQ', 'IBM', 'INTC', 'KO', 'MCD', 'NOV', 'ORCL', 'QCOM', 'SLB', 'T', 'UTX', 'VZ', 'WMT'])


Seems Heinz may have changed year end. A favorite they had a fifth quarter in 2014 see 20140320_1_HNZ.txt

Also for LOW there is a typo of the year in their duplicates, so they also have an equivalent missing.

There was also a problem with a number of the 2012_4 files which were reproduced, so need to be back dated.


### Get unique stock identifiers from the PDF file names collected from Bloomberg

Check to see that we have both transcripts and returns for all the stocks in our universe. 

In [36]:
# Collect unique codes
unique_codes = []
for i in TextFiles:
    code = i.split('_')[3].split('.')[0]
    if code not in unique_codes:
        unique_codes.append(code.upper())

print (sorted(unique_codes))
print (len(unique_codes))    

['0848680D', 'AA', 'AAPL', 'ABBV', 'ABT', 'ACN', 'AEP', 'AGN', 'AIG', 'ALL', 'AMGN', 'AMZN', 'AVP', 'AXP', 'BA', 'BAC', 'BAX', 'BHI', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CPB', 'CSCO', 'CVS', 'CVX', 'DD', 'DELL', 'DHR', 'DIS', 'DOW', 'DUK', 'DVN', 'EMC', 'EMR', 'ETR', 'EXC', 'F', 'FB', 'FCX', 'FDX', 'FOXA', 'GD', 'GE', 'GILD', 'GM', 'GOOGL', 'GS', 'HAL', 'HD', 'HNZ', 'HON', 'HPQ', 'HSH', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MON', 'MRK', 'MS', 'MSFT', 'NEE', 'NKE', 'NOV', 'NSC', 'NYX', 'ORCL', 'OXY', 'PCLN', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RF', 'RTN', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TWX', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'UTX', 'V', 'VZ', 'WBA', 'WFC', 'WMB', 'WMT', 'WY', 'XOM', 'XRX']
122


#### Comment

There have been 126 codes in the S&P 100 over our back-test window. We don't have transcripts for the following four codes and the reasons why are:

- **BRK/B:** This is the ticker in the S&P 100. Maybe the transcripts are under BRK/A.
- **FOX:** This is a duplicate of FOXA, i.e. there are two share categories, so there is only one transcript across both.
- **DWDP:** There was merger between DOW and Dupont just before the end of our back-test window, so there is not a transcript for the new merger company in our universe.
- **GOOG:** This is a duplicate of GOOGL, i.e. there are two share categories, so there is only one transcript across both.

So really only BRK/B. This won't make or break our analysis.

# Exploratory Analysis

### Check the transcript list indices for each of the major sections of the calls

This section is designed check how well we can parse the transcripts into each of the major sections allowing for differences in the possible headers used.

In [37]:
# Different possible section headers
CO_PART_HEADERS = ['Company Participants']
OTH_PART_HEADERS = ['Other Participants']
MD_SECTION_HEADERS = ['MANAGEMENT DISCUSSION SECTION', 'Presentation']
QA_SECTION_HEADERS = ['Q&A', 'Questions And Answers', 'QUESTION AND ANSWER SECTION', 'QUESTION AND ANSWER SESSION']
DISCLAIMER = ['This transcript may not be 100 percent accurate ']


In [40]:
# New text file format has headers
def remove_header_footers(ts):
    i = 0
    header_footer = []
    while ts[i].find('Page') == -1:
        if ts[i].find('Fixed') != -1:
            print (ts[1], ts[2])
        header_footer.append(ts[i])
        i += 1
    pages = ts[i]
    pages = pages.split(' ')
    page_num = int(pages[-1])
    for i in range(1, page_num + 1):
        header_footer.append('Page ' + str(i) + ' of ' + str(page_num))
    ts = [line for line in ts if line not in header_footer]
    return ts


# Read a transcript into a list
def read_transcript(file_path):
    # read in all lines of the transcript
    with open(file_path, "r") as file:
        try:
            ts = [line.rstrip(' \n').rstrip('\n').rstrip(' \r').rstrip('\r') for line in file] # remove new line and any surrounding spaces
            ts = remove_header_footers(ts)
        except UnicodeDecodeError:
            ts = ["UnicodeDecodeError in file"]
    return ts

# Find the index value of a full string from a list of possible strings
def find_full_string_index(ts, string_list):
    result = -1
    for i in range(len(string_list)):
        if string_list[i] in ts:
            result = ts.index(string_list[i])
    return result

# Find the index of the disclaimer
def find_disclaimer_index(ts, string_list):
    result = -1
    k = len(ts) - 1
    while True:
        for my_str in string_list:
            if ts[k].find(my_str) != -1:
                result = k
                break
        k -= 1
        if k < 0:
            break
    return result

# Get transcript list indices from a single transcript
def get_basic_parameters(ts):
    call_type = ts[0]
    co_parts = find_full_string_index(ts,CO_PART_HEADERS)
    oth_parts = find_full_string_index(ts,OTH_PART_HEADERS)
    md = find_full_string_index(ts,MD_SECTION_HEADERS)
    qa = find_full_string_index(ts,QA_SECTION_HEADERS)
    disc = find_disclaimer_index(ts,DISCLAIMER)
    return [call_type, co_parts, oth_parts, md, qa, disc]


In [41]:
# Print results that have a -1 in the index parameters
# Broken up by Q&A problem or other problem
no_QA = []
print ('Problems other than missing Q&A:')
for path in TextFiles:
    ts = read_transcript(path)
    params = get_basic_parameters(ts)
    # Check if Q&A problem
    if params[2] == -1 or params[4] == -1:
        no_QA.append((path, params)) 
    # Check if other problem
    if params[1] == -1 or params[3] == -1 or params[5] == -1:
        print (path, params)

print ()
print ('Calls with Q&A problem:')
i = 0
for call in no_QA:
    if call[1][2] == -1 and call[1][4] == -1:
        pass
        # print ('No Q&A:', call[0])
    else:
        print ('Other Q&A problem:', call[0], call[1])
        i += 1
print ()
print ('Total calls with no Q&A:', len(no_QA) - i)
print ('Total calls with Q&A problems other than no Q&A:', i)


Problems other than missing Q&A:
('Company Ticker: C US', 'Date: 2010-10-22')
('Company Ticker: F US', 'Date: 2010-10-26')
('Company Ticker: C US', 'Date: 2011-01-21')
('Company Ticker: F US', 'Date: 2011-04-26')
('Company Ticker: C US', 'Date: 2011-04-27')
('Company Ticker: C US', 'Date: 2011-07-21')
('Company Ticker: AXP US', 'Date: 2011-07-28')
('Company Ticker: C US', 'Date: 2011-10-20')
('Company Ticker: C US', 'Date: 2012-01-24')
('Company Ticker: C US', 'Date: 2012-04-25')
('Company Ticker: F US', 'Date: 2012-04-27')
('Company Ticker: AXP US', 'Date: 2012-07-19')
('Company Ticker: C US', 'Date: 2012-07-20')
('Company Ticker: GS US', 'Date: 2012-07-24')
('Company Ticker: F US', 'Date: 2012-07-25')
('Company Ticker: C US', 'Date: 2012-10-24')
('Company Ticker: F US', 'Date: 2012-10-30')
('Company Ticker: GS US', 'Date: 2013-01-23')
('Company Ticker: C US', 'Date: 2013-01-25')
('Company Ticker: F US', 'Date: 2013-01-29')
('Company Ticker: C US', 'Date: 2013-04-22')
('Company Ticker

### Comment
Main problems are:

(1) Sometime there is no Q&A section in a call. This can be commonly identified because no index can be found for both `Other Participants` and the `Q&A` section.

(2) Some of the second files appear to be GAAP presentations.

(3) Probably also just want the Earnings Calls only, i.e. exclude Analyst Day, Fixed Income, Pre-recorded, etc.?



### Store a transcript for testing

In [42]:
# Store a test transcript
i = 2
ts = read_transcript(TextFiles[i])
TextFiles[i]


'/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/T1/20100730_1_MET.txt'

In [43]:
ts

['Q2 2010 Earnings Call',
 'Company Participants',
 '\x95 Conor Murphy, Investor Relations',
 '\x95 C. Robert Henrikson, Chairman, President and Chief Executive Officer',
 '\x95 Steven A. Kandarian, Executive Vice President and Chief Investment Officer',
 '\x95 William J. Wheeler, Executive Vice President and Chief Financial Officer',
 '\x95 William Mullaney',
 '\x95 William Toppeta',
 '\x95 Peter Carlson',
 'Other Participants',
 '\x95 Mark Finkelstein',
 '\x95 John Hall',
 '\x95 Colin Devine',
 '\x95 Jimmy Bhullar',
 '\x95 John Nadel',
 '\x95 Suneet Kamath',
 '\x95 Andrew Kligerman',
 '\x95 Randy Binner',
 'MANAGEMENT DISCUSSION SECTION',
 'Operator',
 'Ladies and gentlemen, thank you for standing by. Welcome to the MetLife second quarter earnings release conference',
 'call. [Operator Instructions] As a reminder, this conference is being recorded.',
 'Before we get started, I would like to read the following statement on behalf of MetLife.',
 'Except with respect to historical infor

### Check for items that need to be excluded from the strings because they are housed in square brackets '[]'

It is notable that some text that is not spoken is included in the transcripts between square brackets '[]', so this should ideally be removed as part of pre-processing.

The following lines in the test transcript feature square brackets:

In [44]:
# find examples with squares brackets
[line for line in ts if '[' in line]


['call. [Operator Instructions] As a reminder, this conference is being recorded.',
 '[Operator Instructions] We have our first question from the line of Mark Finkelstein with Macquarie. Please go ahead,',
 'the situation, including the National Organization of [Health and] Life Guaranty Association [NOHLGA], the NAIC,',
 '[Laughter]',
 '[Accumulated Other Comprehensive Income] and the movements there and also, net income versus operating income.']

We can use regex to strip out these occurrences:

In [45]:
# Use regex to replace characters in between and including square brackets
[re.sub(r'\[.+?\]', '', line) for line in ts if '[' in line]

# Note that the above makes no allowance for spaces on either side of the square brackets
# The following makes some allowance, but seems to be hard to allow for all possibilities neatly
#[re.sub(r'\[.+?\]\s*', '', re.sub(r'\[.+?\]\.\s*', '', i)) for i in a]

['call.  As a reminder, this conference is being recorded.',
 ' We have our first question from the line of Mark Finkelstein with Macquarie. Please go ahead,',
 'the situation, including the National Organization of  Life Guaranty Association , the NAIC,',
 '',
 ' and the movements there and also, net income versus operating income.']

### Pre-process function to remove contents of square brackets

Use the above as a pre-processing function:

In [46]:
def remove_square_brackets(my_str):
    return re.sub(r'\[.+?\]', '', my_str)

def remove_square_brackets_transcript(ts):
    return list(map(lambda line: remove_square_brackets(line), ts))


Confirm that all occurrences have been removed

In [47]:
ts_test = remove_square_brackets_transcript(ts)
[re.sub(r'\[.+?\]', '', line) for line in ts_test if '[' in line]

[]

### Get Company Participants

Sometimes the names are used with their title, without their title and without their middle initial. Using the indices for the `Company Participants` and `Other Participants` section the `get_parts` function retrieves the participants from the Company Participants section and creates a set of all these possible name combinations.

**Note:** We possibly don't need the names because we are more interested in aggregate evasiveness, but some papers show that different roled people can behave with systematically different behaviour, so could be something we test if time permits.


In [48]:
# Find participants on the call
def get_parts(ts, start_index, end_index):
    result = [name[re.search("[a-z]", name.lower()).start():] for name in ts[start_index+1:end_index] if len(name) >= 3]
    # Check for problematic names
    extra_names = []
    for name in result:
        # Find any title after name and remove it
        if name.find(',') != -1:
            name_only = name[:name.find(',')]
            extra_names.append(name_only)
        # Find names with middle initials and remove them
        elif name.find('.') != -1:
            name_only = remove_middle_initials(name)
            if name_only != name:
                extra_names.append(name_only)
    # Find names in extra names with middle initials and remove them
    for name in extra_names:
        if name.find('.') != -1:
            name_only = remove_middle_initials(name)
            if name_only != name:
                extra_names.append(name_only)
    return result + extra_names

# Remove middle initial from a name
def remove_middle_initials(my_str):
    # count = 0
    while True:
        if my_str.find('.') != -1:
            new_my_str = re.sub(r'(\s)([A-Z].)(\s)', r"\1", my_str)
            if new_my_str == my_str:
                break
            else:
                my_str = new_my_str
            # count += 1
            # if count >= 10:
            #     break
        else:
            break
    return my_str


In [49]:
# Get Company Participants
start = find_full_string_index(ts,CO_PART_HEADERS)
end = find_full_string_index(ts,OTH_PART_HEADERS) - 1
co_part_names = get_parts(ts, start, end)
co_part_names

['Conor Murphy, Investor Relations',
 'C. Robert Henrikson, Chairman, President and Chief Executive Officer',
 'Steven A. Kandarian, Executive Vice President and Chief Investment Officer',
 'William J. Wheeler, Executive Vice President and Chief Financial Officer',
 'William Mullaney',
 'William Toppeta',
 'Conor Murphy',
 'C. Robert Henrikson',
 'Steven A. Kandarian',
 'William J. Wheeler',
 'Steven Kandarian',
 'William Wheeler']

**Comment:** This appears to be working quite well.

### Get Other Participants

Other Participants tend to be identified by name only, but if any titles or middle initials are used the `get_parts` function will take the participants from the Other Participants section and creates a set of all these possible name combinations.

**Note:** It is unlikely that we will need individual analyst names because we are interested in management evasiveness.


In [50]:
# Get Other Participants
start = find_full_string_index(ts, OTH_PART_HEADERS)
end = find_full_string_index(ts, MD_SECTION_HEADERS) - 1
oth_part_names = get_parts(ts, start, end)
oth_part_names


['Mark Finkelstein',
 'John Hall',
 'Colin Devine',
 'Jimmy Bhullar',
 'John Nadel',
 'Suneet Kamath',
 'Andrew Kligerman']

**Comment:** This appears to be working quite well.

### Get an indexed section of the call

In [51]:
# Find a section of the call
def get_section(ts, start_index, end_index):
    result = ts[start_index+1:end_index]
    result = [line for line in result if line not in ['\x0c']]   # clean the transcript for unnecessary lines
    return result


### Get MD Section


In [52]:
def get_MD(ts, names, start_index, end_index):

    curr_speaker = ""
    section_header = "MD"
    
    # Create an empty dict for comments by each manager
    pts = {name: "" for name in names}

    # Get the required section
    section = get_section(ts, start_index, end_index)
    
    # Find the first instance of a co_parts speaker
    start_section = 0
    while True:
        if section[start_section] not in names:
            start_section += 1
        else:
            break
        if start_section == len(section):
            start_section = -1
            break

    if start_section == -1:
        pts['No speaker found'] = section_header
    else:
        # Reduce MD Section to management speaking only
        section = section[start_section:]
        # populate comment dict
        for line in section:
            if line in names:
                curr_speaker = line
            elif len(line) != 0:
                pts[curr_speaker] += line + " "

    return pts


In [282]:
# Get MD Section
start = find_full_string_index(ts,MD_SECTION_HEADERS)
end = find_full_string_index(ts,QA_SECTION_HEADERS) - 1
get_MD(ts, co_part_names, start, end)

{'Jeanette Ourada': '',
 'Jeanette Ourada, General Manager, Investor Relations': "Thanks, Pat. Turning to Slide 4, I will compare results of the second quarter 2010 with the first quarter 2010. As a reminder, our earnings release compares second quarter 2010 with the same quarter a year ago. Second quarter earnings increased about 850 million from the first quarter. Upstream earnings were 180 million lower due to unfavorable tax charges and higher OpEx, partially offset by a favorable foreign currency variance. Second quarter downstream results saw significant improvement of nearly 780 million. Earnings benefited from stronger U.S. margins, favorable foreign currency and timing variances, as well as the absence of first quarter charges related to planned employee reductions. The other bar largely reflects a favorable swing in corporate tax items and lower corporate charges. On Slide 5, our U.S. upstream earnings for the second quarter were 66 million lower than the first quarter's resu

### QA Section


#### Raw review of the QA Section

In [53]:
start = find_full_string_index(ts, QA_SECTION_HEADERS)
end = find_disclaimer_index(ts,DISCLAIMER)
get_section(ts, start, end)

['Operator',
 '[Operator Instructions] We have our first question from the line of Mark Finkelstein with Macquarie. Please go ahead,',
 'your line is open.',
 "<Q - Mark Finkelstein>: Good morning, hopefully you can hear me okay. There's a lot of static. I guess it was a good",
 'quarter. You had favorable mortality, good morbidity, good investment results, good expense control across the board.',
 "I think if you looked at a core number, it's somewhere in the 1.10 to 1.15 range. I guess what I'm really interested in is",
 'how should we think about this level of earnings and whether we should be trending off this number or whether we',
 'should be thinking about a little bit lower number just given the level of favorability in the quarter.',
 "<A - William J. Wheeler, Executive Vice President and Chief Financial Officer>: Hi, Mark, it's Bill. Yes, we did",
 "hear your question. I always hesitate say whether $1.11 is a good run rate or not, but I'll give you some sense of the",
 'pluse

#### Get Questions with their matching Answers (with no consideration of who the respondent is)

This will be useful for the topic modelling part to determine whether the answer is on topic with the question.

Some considerations allowed for:

- Some of the strings in the transcript are NULL's, i.e. `''`, so these are ignored

- The Operator makes comments after a line that is solely `'Operator'`. Any comments after such a line are ignored until a question is found.

> For example, in the following portion of a call we want to ignore the Question by Doug Legatte that is a concluding statement and the comments by the Operator. The current code acheives this:

> `
 '<Q - Doug Leggate>: Okay, Pat. Would you give us a run rate perhaps for the underlying tax rate?',
 '',
 '<A - Patricia E. Yarrington, Chief Financial Officer & Vice President>: I think the best way to look at it is, over a',
 "long period of time, we've been a little bit lower this year than we were in 2009. So maybe on average for 2010 we may",
 "be a little bit lower. But it's not an area I want to get into predicting.",
 '',
 '<Q - Doug Leggate>: All right. Thanks for answering my questions. Thanks.',
 '',
 'Operator',
 '',
 'Our next question comes from Evan Calio with Morgan Stanley. Please go ahead.',
 '',
 '<Q - Evan Calio>: Good morning, everybody.',
`

- Some strings commmence with `'<Q'` or `'<A'`, but have no participant associated with them, so the next character in the string is `'>'`, whilst others have an associated participant designated by the next 3 characters being `' - '` before their name then `'>'`.







In [54]:
# This procedure combines the raw Q&A section into pairs of questions and answers in a dictionary

def get_QA(ts, start_index, end_index):

    pts = {}
    # get the Q&A section
    qa_section = get_section(ts, start_index, end_index)

    q_or_a = None
    currQ = ''
    currA = ''
    
    for line in qa_section:
        line = remove_square_brackets(line)
        if len(line) > 0:                                     # if line is '' then ignore otherwise process the line
            if line == 'Operator':                            # if line is 'Operator' then set everything NULL and ignore
                q_or_a = None
                currQ = ''
                currA = ''
            if line[0] == "<":                                # if Q or A found do something
                if line[0:2] == "<Q":
                    if q_or_a == 'A':                         # if Q found and q_or_a is not NULL then add Q: A to the dictionary
                        pts[currQ[:-1]] = currA[:-1]          # and reset the parameters to NULL
                        currQ = ''
                        currA = ''
                        q_or_a = None    
                    end_qa_tag = line.find(">")
                    currQ += line[end_qa_tag + 3:] + " "      # Concatenate the string after the '>' to the previous string
                    q_or_a = 'Q'                              # Given Q found set q_or_a = 'Q'
                if line[0:2] == "<A":
                    end_qa_tag = line.find(">")             
                    currA += line[end_qa_tag + 3:] + " "      # Concatenate the string after the '>' to the previous string
                    q_or_a = 'A'                              # Given Q found set q_or_a = 'Q'
            else:                                             
                if q_or_a is not None:                        # Other if currently a Q or A just concatenate the string 
                    if q_or_a == 'Q':
                        currQ += line + " "
                    else:
                        currA += line + " "

    if currQ != '' and currA != '':                           # Add last Q: A pair to the dictionary if necessary
        pts[currQ[:-1]] = currA[:-1]

    return pts




In [55]:
start = find_full_string_index(ts, QA_SECTION_HEADERS)
end = find_disclaimer_index(ts,DISCLAIMER)
get_QA(ts, start, end)

{'Good morning, everyone.': 'Good morning, John.',
 "Good morning, hopefully you can hear me okay. There's a lot of static. I guess it was a good quarter. You had favorable mortality, good morbidity, good investment results, good expense control across the board. I think if you looked at a core number, it's somewhere in the 1.10 to 1.15 range. I guess what I'm really interested in is how should we think about this level of earnings and whether we should be trending off this number or whether we should be thinking about a little bit lower number just given the level of favorability in the quarter.": "Hi, Mark, it's Bill. Yes, we did hear your question. I always hesitate say whether $1.11 is a good run rate or not, but I'll give you some sense of the pluses and minuses. You're right, underwriting was very strong, and so I think a lot of that is sustainable. Some of it was probably just good fortune. But I also think, by the way, like the improvements in dental underwriting margin are ver

#### Check if 'Operator' has been successfully removed from the questions in the Q&A section

In [56]:
# This should return an empty set if no mention of 'Operator'
[k for k in get_QA(ts, start, end).keys() if 'Operator' in k]


[]

#### Check if 'Operator' has been successfully removed from the answers in the Q&A section

In [57]:
[v for v in get_QA(ts, start, end).values() if 'Operator' in v]


[]

#### Examples of 'but' in the answers

In [58]:
[v for v in get_QA(ts, start, end).values() if ', but' in v]


["Yes, I'm going to give that to Mr. Wheeler. Bill only gets so much air time each call. We have to limit it. With the Japanese, obviously we explained when we announced Alico that under the general rules, we thought we wouldn't be able to take out regular dividends out of Japan until the 2012 timeframe. It's quite possible that we may get some flexibility on that from the regulator, and we've had some \x96 we haven't really brought that up in great detail with them yet because we obviously have been doing some other things with them about discussing other issues with them at the moment. But I'm hopeful we might make some progress on that front earlier, but there's nothing definitive to say now.",
 "It's Bill Mullaney. I'll talk about the first two parts of your question. As it relates to the U.K. closeout business, we had lower U.K. closeout sales this quarter than we had in the first quarter. As we've talked about in the past, closeout sales are lumpy and they do vary from quarter to


### Data Cleaning

Data cleaning is absolutely crucial for generating a useful topic model: as the saying goes, “garbage in, garbage out.” The steps below are common to most natural language processing methods:

- Tokenizing: converting a document to its atomic elements.

- Stopping: removing meaningless words.

- Stemming: merging words that are equivalent in meaning.


### Bespoke cleaning

The following procedures deal with strings with words that have apostrophes, hyphens, slashes and unusual characters in them to help best isolate actual words.

In [106]:
def short_form_replace(my_str):
    """Convert apostrophes in known short-forms in my_string to long-forms"""
    my_str = re.sub(r'let\'s', "let us", my_str)
    my_str = re.sub(r'Let\'s', "let us", my_str)
    my_str = re.sub(r'won\'t', "will not", my_str)
    my_str = re.sub(r'can\'t', "cannot", my_str)
    my_str = re.sub(r'shan\'t', "shall not", my_str)
    my_str = re.sub(r'Won\'t', "Will not", my_str)
    my_str = re.sub(r'Can\'t', "Cannot", my_str)
    my_str = re.sub(r'Shan\'t', "Shall not", my_str)
    my_str = re.sub(r'n\'t', " not", my_str)
    my_str = re.sub(r'\'ve', " have", my_str)
    my_str = re.sub(r'\'re', " are", my_str)
    my_str = re.sub(r'\'m', " am", my_str)
    my_str = re.sub(r'\'ll', " will", my_str)
    my_str = re.sub(r'\'d', " would", my_str)     # note could also be did or had as well
    my_str = re.sub(r'it\'s', "it is", my_str)
    my_str = re.sub(r'he\'s', "he is", my_str)
    my_str = re.sub(r'she\'s', "she is", my_str)
    my_str = re.sub(r'that\'s', "that is", my_str)
    my_str = re.sub(r'what\'s', "what is", my_str)
    my_str = re.sub(r'here\'s', "here is", my_str)
    my_str = re.sub(r'there\'s', "there is", my_str)
    my_str = re.sub(r'who\'s', "who is", my_str)
    my_str = re.sub(r'It\'s', "It is", my_str)
    my_str = re.sub(r'He\'s', "He is", my_str)
    my_str = re.sub(r'She\'s', "She is", my_str)
    my_str = re.sub(r'Shat\'s', "That is", my_str)
    my_str = re.sub(r'What\'s', "What is", my_str)
    my_str = re.sub(r'Here\'s', "Here is", my_str)
    my_str = re.sub(r'There\'s', "There is", my_str)
    my_str = re.sub(r"Who's", "Who is", my_str)
    return my_str


def apostrophe_s_replace(my_str):
    """Remove 's at the end of words in my_string. Best to run this after short_form_replace()"""
    my_str = re.sub(r"([A-z])\'s", r"\1", my_str)
    return my_str


def hyphen_replace(my_str):
    """Replace hyphens in hyphenated words with a space"""
    my_str = re.sub(r"([A-z])(\-)([A-z])", r"\1 \3", my_str)
    my_str = re.sub(r"([A-z])(\-)(\s)([A-z])", r"\1 \4", my_str)
    return my_str


def forward_slash_replace(my_str):
    """Replace forward slashes in combined words with a space."""
    my_str = re.sub(r"([A-z])(/)([A-z])", r"\1 \3", my_str)
    my_str = my_str.replace(')/', ' ')
    my_str = my_str.replace('/(', ' ')
    return my_str


def replace_apostrophes(my_str):
    """Tidy up my_string for all apostrophe-related issues."""
    my_str = short_form_replace(my_str)
    my_str = apostrophe_s_replace(my_str)
    return my_str

def replace_x96(my_str):
    """Tidy up my_string for all apostrophe-related issues."""
    my_str = my_str.replace('\x96', '')
    return my_str



def clean_text(my_str):
    """Clean my_string for all known string issues to prepare for text analysis"""
    my_str = replace_apostrophes(my_str)
    my_str = hyphen_replace(my_str)
    my_str = forward_slash_replace(my_str)
    mystr = replace_x96(my_str)
    return my_str


#### Example

In [107]:
for item in test_strings:
    print ('Input string:', item)
    print
    print ('Output string:', clean_text(item))
    print

('Input string:', "Okay? Thanks, Don! Next-slide is a 493 little bit of detail on the significant charges, with the Pizza Sprint and German acquisitions. As you can see, we've got the split now between Pizza Sprint and Germany.")

('Output string:', 'Okay? Thanks, Don! Next slide is a 493 little bit of detail on the significant charges, with the Pizza Sprint and German acquisitions. As you can see, we have got the split now between Pizza Sprint and Germany.')

('Input string:', 'If not staying at the XXXX please note that East Malvern footy club is right at Darling station (see train line map below). To make it easy for yourself to get around by train it is recommended that you stay somewhere close to a train station that is on the Glen Waverley train line (the full train line is shown below). From there you will be able to easily get to and from, East Malvern Knights Footy Club (Darling station), Etihad (Southern Cross station) and the MCG (Richmond or Parliament stations).')

('Outpu


### Tokenization

Tokenization segments a document into its atomic elements. In this case, we are interested in tokenizing to words. Tokenization can be performed many ways–we are using NLTK’s `tokenize.regexp` module

In [65]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

The above code will match any word characters until it reaches a non-word character, like a space. This is a simple solution, but can cause problems for words like “don’t” which will be read as two tokens, “don” and “t.” Which is why there is `clean_text()` (above), i.e. to tidy strings for primary within-word punctuation marks.

NLTK also provides a number of pre-constructed tokenizers like `nltk.tokenize.simple`. For unique use cases, it’s better to use regex and iterate until your document is accurately tokenized.

An alternate is to: 

In [292]:
# Remove the punctuation using the character deletion step of translate
# no_punctuation = lowers.translate(None, string.punctuation)
# tokens = nltk.word_tokenize(no_punctuation)

#### Example

In [66]:
# Example test string
raw = test_string.lower()
short_form_adjusted = clean_text(raw) # allow for certain primary punctuation mark adjustments (i.e. ', - and /)
tokens = tokenizer.tokenize(short_form_adjusted)

In [67]:
print (tokens)

['okay', 'thanks', 'don', 'next', 'slide', 'is', 'a', '493', 'little', 'bit', 'of', 'detail', 'on', 'the', 'significant', 'charges', 'with', 'the', 'pizza', 'sprint', 'and', 'german', 'acquisitions', 'as', 'you', 'can', 'see', 'we', 'have', 'got', 'the', 'split', 'now', 'between', 'pizza', 'sprint', 'and', 'germany']


### Stop Words

Certain parts of English speech, like conjunctions (“for”, “or”) or the word “the” are meaningless to a topic model. These terms are called stop words and need to be removed from our token list.

The definition of a stop word is flexible and the kind of documents may alter that definition. For example, if we’re topic modeling a collection of music reviews, then terms like “The Who” will have trouble being surfaced because “the” is a common stop word and is usually removed. You can always construct your own stop word list or seek out another package to fit your use case.

We can use words from the `stop_words` package from Pypi, a **relatively conservative** list to create a list of stop words:

In [68]:
# Import stop words list
stop_words = get_dictionary(dict_path,'stop_words.txt')

In [69]:
print (stop_words)

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 't

Alternately we can use the stop words in `nltk`:

In [70]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Richard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [71]:
from nltk.corpus import stopwords

In [72]:
# Create English stop words list from NLTK
nltk_stop_words = stopwords.words('english')

In [73]:
print (nltk_stop_words)

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

Check the difference between the two:

In [74]:
print ([i for i in nltk_stop_words if i not in stop_words])

[u's', u't', u'can', u'will', u'just', u'don', u'now', u'd', u'll', u'm', u'o', u're', u've', u'y', u'ain', u'aren', u'couldn', u'didn', u'doesn', u'hadn', u'hasn', u'haven', u'isn', u'ma', u'mightn', u'mustn', u'needn', u'shan', u'shouldn', u'wasn', u'weren', u'won', u'wouldn']


Seems to include a few extras that might be useful, so combine as per below. Need to check though. For example, the word 'won' might be important in finance.

In [75]:
stop_words = sorted(list(set(stop_words + nltk_stop_words)))
print (stop_words)

['a', 'about', 'above', 'after', 'again', 'against', u'ain', 'all', 'am', 'an', 'and', 'any', 'are', u'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', u'can', "can't", 'cannot', 'could', u'couldn', "couldn't", u'd', 'did', u'didn', "didn't", 'do', 'does', u'doesn', "doesn't", 'doing', u'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', u'hadn', "hadn't", 'has', u'hasn', "hasn't", 'have', u'haven', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', u'isn', "isn't", 'it', "it's", 'its', 'itself', u'just', "let's", u'll', u'm', u'ma', 'me', u'mightn', 'more', 'most', u'mustn', "mustn't", 'my', 'myself', u'needn', 'no', 'nor', 'not', u'now', u'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'o

#### Example

In [76]:
# Remove stop words from tokens
stopped_tokens = [i for i in tokens if not i in stop_words]

In [77]:
print (stopped_tokens)

['okay', 'thanks', 'next', 'slide', '493', 'little', 'bit', 'detail', 'significant', 'charges', 'pizza', 'sprint', 'german', 'acquisitions', 'see', 'got', 'split', 'pizza', 'sprint', 'germany']


### Stemming

Stemming words is another common NLP technique to reduce topically similar words to their root. Stemming is important for topic modeling, which would otherwise view those terms as separate entities and reduce their importance in the model.

Like stopping, stemming is flexible and some methods are more aggressive. The Porter stemming algorithm is the most widely used method. To implement a Porter stemming algorithm, import the Porter Stemmer module from NLTK:

In [78]:
from nltk.stem.porter import PorterStemmer

#### Example

In [79]:
# Create porter_stemmer of class PorterStemmer
porter_stemmer = PorterStemmer()
# stem the stopped tokens
stemmed_tokens = [porter_stemmer.stem(i) for i in stopped_tokens]

In [80]:
print (stemmed_tokens)

['okay', u'thank', 'next', 'slide', '493', u'littl', 'bit', 'detail', u'signific', u'charg', 'pizza', 'sprint', 'german', u'acquisit', 'see', 'got', 'split', 'pizza', 'sprint', u'germani']


### Vectorize the word list counts

Need to establish how similar the usage of the function words are between the MD Section and the Q&A section.

To do so we need to vectorize the MD Section word counts of the function words of interest to determine their typical frequencies and then compare them to the usage in each of the answers given to questions.


In [81]:
def vectorize_word_list_counts(words, word_list):
    """Count the frequency of words in a document from a word list"""
    count_vector = [words.count(word) for word in sorted(word_list)]
    total_count = sum(count_vector)
    if total_count != 0:
        count_vector = [float(count)/total_count for count in count_vector]
    return count_vector


In [82]:
count_vector = vectorize_word_list_counts(stemmed_tokens, function_words)

In [83]:
print (count_vector)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

#### Concerns

- Need to include stop words in the base line analysis because as many of them are in the function words list (see below).

- Need to possibly give more weight to longer answers or possibly screen out answers less than a certain lenght because some shorted answers are not answers to commerical questions.

In [84]:
# Words appearing in the function and stop word lists
print ([i for i in stop_words if i in function_words])

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', u'can', 'cannot', 'could', 'did', 'do', 'does', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'itself', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', u'now', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', u're', 'same', 'she', 'should', 'so', 'some', 'such', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', u'will', 'with', 'would', 'you', 'your', 'yours', 'yourself', 'yourselves']


### Scriptedness pre-processing a document

In [117]:
def scriptedness_pre_process_document(doc):
    tokenizer = RegexpTokenizer(r'\w+')  # This finds letters only and breaks things up at each non-letters character
    porter_stemmer = PorterStemmer()
    lower_doc = doc.lower()
    non_ascii_doc = remove_non_ascii(lower_doc)           
    clean_lower_doc = remove_square_brackets(non_ascii_doc)
    clean_doc = clean_text(clean_lower_doc)
    tokens = tokenizer.tokenize(clean_doc)
    stopped_tokens = tokens # no stopping
    stemmed_tokens = [porter_stemmer.stem(i) for i in stopped_tokens]
    count_vector = vectorize_word_list_counts(stemmed_tokens, function_words)
    return count_vector


In [86]:
# Examples using dummy test documents
MD_scriptedness = [scriptedness_pre_process_document(doc) for doc in doc_set]
QA_scriptedness = [scriptedness_pre_process_document(doc) for doc in test_strings]


In [87]:
print (QA_scriptedness)

[[0.05555555555555555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.05555555555555555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

### Scriptedness makes use of Cosine Similarity

In [88]:
def cosine_similarity(v1, v2):
    """Returns the cosine similarity score for two equal length vectors of numbers.
    To compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)."""
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x, y = v1[i], v2[i]
        sumxx += x * x
        sumyy += y * y
        sumxy += x * y
    if math.sqrt(sumxx * sumyy) !=0:
        result = round(sumxy/math.sqrt(sumxx * sumyy),6)
    else:
        result = 0
    return result


#### Example

In [89]:
v1 = MD_scriptedness[0]
v2 = MD_scriptedness[1]

cosine_similarity(v1, v2)

0.603023

In [90]:
# This gets the average similarity for one comment, i.e. full MD section, and all the answers separately
def get_avg_score(comment, answers):
    scores = []
    v1 = comment
    for v2 in answers:
        scores.append(cosine_similarity(v1, v2))
    avg_score = round(sum(scores)/len(scores),6)
    return avg_score

get_avg_score(MD_scriptedness[0],QA_scriptedness)

0.269484

### Perform Scriptedness Analysis

In [97]:
TextFiles[0]

'/Users/Richard/Desktop/Berkeley/w266/repo/w266_project/T1/20100730_1_AEP.txt'

In [101]:
def get_filename_components(file_path, i):
    file_path = file_path.split('_')
    date = file_path[1][-8:]
    file_num = file_path[2]
    ticker = file_path[3][:-4]
    print (i, (ticker, date, file_num))

    return (ticker, date, file_num)
    

In [102]:
get_filename_components(TextFiles[0], 1)

(1, ('AEP', '20100730', '1'))


('AEP', '20100730', '1')

In [121]:
test = 'oducing � we actually just recently'

def remove_non_ascii(doc):      
    non_ascii = ''.join([char for char in doc if ord(char) in range(128)])
    return non_ascii

remove_non_ascii(test)


'oducing  we actually just recently'

In [122]:
import time
start_time = time.time()

# TextFiles = get_files(input_path,'txt')
# function_words = get_dictionary(dict_path,'function_words.txt')

results = {}
i = 0

for f in TextFiles:
    (ticker, date, file_num) = get_filename_components(f, i)
    qts = quick_read_transcript(f)
    if qts[2].find('Earnings Call') != -1 and qts[2].find('Fixed') == -1:   # Found an earnings call
        if date not in results.keys():
            results[date] = {}
        ts = read_transcript(f)
        params = get_basic_parameters(ts)
        # Check if Q&A problem
        if -1 in params:
            results[date][ticker] = 'NA'
        else:
            # Get Company Participants
            start = find_full_string_index(ts, CO_PART_HEADERS)
            end = find_full_string_index(ts, OTH_PART_HEADERS) - 1
            co_part_names = get_parts(ts, start, end)

            # Get MD Section
            start = find_full_string_index(ts, MD_SECTION_HEADERS)
            end = find_full_string_index(ts, QA_SECTION_HEADERS) - 1
            MD = get_MD(ts, co_part_names, start, end)
            comments = [comment for comment in MD.values() if len(comment) > 20]
            comment = ' '.join(comments)
            if i == 60:
                print (comment)
            comment = scriptedness_pre_process_document(comment)

            # Get QA Section
            start = find_full_string_index(ts, QA_SECTION_HEADERS)
            end = find_disclaimer_index(ts, DISCLAIMER)
            QA = get_QA(ts, start, end)    
           
            # get the answers with a condition to ensure the answer is of at least some length
            answers = [answer for answer in QA.values() if len(answer) > 20]
#             if i == 7:
#                 for answer in answers:
#                     for i in answer:
#                         if ord(i) not in range(128):
#                             print (i, answer)
            answers = [scriptedness_pre_process_document(answer) for answer in answers]

            if len(answers) > 0:
                results[date][ticker] = get_avg_score(comment, answers)
            else:
                results[date][ticker] = 'NA'
                    
        i += 1

print ("--- %s seconds ---" % (time.time() - start_time))

    

(0, ('AEP', '20100730', '1'))
(1, ('CVX', '20100730', '1'))
(2, ('MET', '20100730', '1'))
(3, ('MRK', '20100730', '1'))
(4, ('SPG', '20100730', '1'))
(5, ('WY', '20100730', '1'))
(6, ('BHI', '20100803', '1'))
(7, ('DOW', '20100803', '1'))
(8, ('DUK', '20100803', '1'))
(9, ('EMR', '20100803', '1'))
(10, ('ETR', '20100803', '1'))
(11, ('MA', '20100803', '1'))
(12, ('NYX', '20100803', '1'))
(13, ('PCLN', '20100803', '1'))
(14, ('PFE', '20100803', '1'))
(15, ('PG', '20100803', '1'))
(16, ('CHTR', '20100804', '1'))
(17, ('DVN', '20100804', '1'))
(18, ('FOXA', '20100804', '1'))
(19, ('TWX', '20100804', '1'))
(20, ('AGN', '20100805', '1'))
(21, ('ALL', '20100805', '1'))
(22, ('MDLZ', '20100805', '1'))
(23, ('AIG', '20100806', '1'))
(24, ('DIS', '20100810', '1'))
(25, ('CSCO', '20100811', '1'))
(26, ('GM', '20100812', '1'))
(27, ('HSH', '20100812', '1'))
(28, ('LOW', '20100816', '1'))
(29, ('HD', '20100817', '1'))
(30, ('WMT', '20100817', '1'))
(31, ('TGT', '20100818', '1'))
(32, ('DELL', '201

IndexError: string index out of range

In [468]:
results

{'20100730': {'AEP': 0.700156,
  'CVX': 0.58198,
  'MET': 0.59171,
  'MRK': 0.747433,
  'SPG': 0.60501,
  'WY': 0.532411},
 '20100803': {'BHI': 0.629955,
  'DOW': 0.65646,
  'DUK': 0.578824,
  'EMR': 0.519417,
  'ETR': 0.600502,
  'MA': 0.614056,
  'NYX': 0.51326,
  'PCLN': 0.656028,
  'PFE': 0.360584,
  'PG': 0.336925},
 '20100804': {'CHTR': 0.671091,
  'DVN': 0.625642,
  'FOXA': 0.523457,
  'TWX': 0.739454},
 '20100805': {'AGN': 0.60002, 'ALL': 0.510867, 'MDLZ': 0.590864},
 '20100806': {'AIG': 'NA'},
 '20100810': {'DIS': 0.72788},
 '20100811': {'CSCO': 0.455655},
 '20100812': {'GM': 0.56016, 'HSH': 0.576464},
 '20100816': {'LOW': 0.637366},
 '20100817': {'HD': 0.663928, 'WMT': 'NA'},
 '20100818': {'TGT': 0.744857},
 '20100819': {'DELL': 0.63952, 'HPQ': 0.705189},
 '20100824': {'MDT': 0.673229},
 '20100901': {'HNZ': 0.577725},
 '20100903': {'CPB': 0.460749},
 '20100916': {'FDX': 0.72698, 'ORCL': 0.712935},
 '20100923': {'NKE': 0.583516},
 '20100928': {'WBA': 0.621681},
 '20100930': {'

In [96]:
results

{'epo/w266': {'': 0.629955}}

# LDA Approach

http://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

### Install packages

In [ ]:
!pip install --index-url=http://pypi.python.org/simple/ --trusted-host pypi.python.org gensim

In [ ]:
!pip install --index-url=http://pypi.python.org/simple/ --trusted-host pypi.python.org smart_open

In [ ]:
import gensim

### Constructing a document-term matrix

The result of our cleaning stage is `texts`, a tokenized, stopped and stemmed list of words from a list of documents. 

To generate an LDA model, we need to understand how frequently each term occurs within each document. To do that, we need to construct a document-term matrix with a package called gensim:

In [ ]:
from gensim import corpora, models

The `Dictionary()` function traverses `texts`, assigning a unique integer id to each unique token while also collecting word counts and relevant statistics. 

In [ ]:
dictionary = corpora.Dictionary(texts)

dictionary

To see each token’s unique integer id, try `print dictionary.token2id`.

In [ ]:
print (dictionary.token2id)

Next, our dictionary must be converted into a bag-of-words:

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]

The doc2bow() function converts dictionary into a bag-of-words. The result, corpus, is a list of vectors equal to the number of documents. In each document vector is a series of tuples. The tuples are (term ID, term frequency) pairs, so if print(dictionary.token2id) says brocolli’s id is 0, then the first tuple indicates that brocolli appeared twice in the first document. doc2bow() only includes terms that actually occur: terms that do not occur in a document will not appear in that document’s vector.

In [ ]:
print (corpus)

In [ ]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

The `LdaModel` class is described in detail in the gensim documentation.

https://radimrehurek.com/gensim/models/ldamodel.html

Parameters used in our example:

- `num_topics`: required. An LDA model requires the user to determine how many topics should be generated. Our document set is small, so we’re only asking for three topics.

- `id2word`: required. The `LdaModel` class requires our previous dictionary to map ids to strings.

- `passes`: optional. The number of laps the model will take through corpus. The greater the number of passes, the more accurate the model will be. A lot of passes can be slow on a very large corpus.

### Examining the results

Our LDA model is now stored as `ldamodel`. We can review our topics with the `print_topic` and `print_topics` methods:

In [ ]:
print(ldamodel.print_topics(num_topics=3, num_words=3))

### Pre-process for LDA

In [ ]:
def LDA_pre_process_documents(documents):
    processed_documents = []
    tokenizer = RegexpTokenizer(r'\w+')  # This finds letters only and breaks things up at each non-letters character
    en_stop = get_stop_words('en')
    porter_stemmer = PorterStemmer()
    for doc in documents:
        lower_doc = doc.lower()
        short_form_extended = clean_text(lower_doc)
        tokens = tokenizer.tokenize(short_form_extended)
        stopped_tokens = tokens # no stopping
        stopped_tokens = [i for i in tokens if not i in en_stop]
        stemmed_tokens = [porter_stemmer.stem(i) for i in stopped_tokens]
        processed_documents.append(stemmed_tokens)
    return processed_documents


In [ ]:
MD_LDA = LDA_pre_process_documents(doc_set)
QA_LDA = LDA_pre_process_documents(test_strings)
texts = MD_LDA + QA_LDA
print texts

### Old code

In [ ]:
# READ IN DICTIONARIES

def get_full_dict():
    """Read in the full McDonald and Loughran word list dictionary as a list"""
    file_path = dict_path + dict_name
    with open(file_path, "r") as file:
        words = [word.lower().rstrip('\n').split(',') for word in file]
    words_only = []
    for i in range(len(words)):
        words_only.append(words[i][0])
    return words_only


def get_bytes_replace_dict():
    """Read in the Non-UTF8 character dictionary as a dict with key=byte character, value=utf-8 character"""
    file_path = dict_path + bytes_dict_name
    with open(file_path, "r") as file:
        words = [word.rstrip('\n').split('\t') for word in file]
    bytes_replace_dict = {}
    for i in range(1, len(words)):
        bytes_replace_dict[words[i][1]] = words[i][2]
    return bytes_replace_dict


def get_result_file(result_name):
    """Result_name is a result file that is a '#' separated file"""
    file_path = dict_path + result_name
    with open(file_path, "r") as file:
        results = [word.rstrip('\n').split('#') for word in file]
    return results


# BASIC STRING FUNCTIONS

def remove_punctuation(my_string):
    """Remove all ASCII punctuation from my_string"""
    # NEED TO INCLUDE NO-UTF8 apostrophe replace
    my_str = short_form_replace(my_string)  # replace short forms with long forms before removing punctuation
    # my_str = ''.join(l for l in my_str if l not in string.punctuation)
    my_punc_remove = "-"
    my_punc = ''.join(l for l in string.punctuation if l not in my_punc_remove)
    my_str = ''.join(l for l in my_str if l not in my_punc)
    my_str = ' '.join(my_str.split())     # this changes all white spaces to single spaces
    return my_str


def remove_digits(my_string):
    """Remove all ASCII digits from my_string"""
    my_str = ''.join(l for l in my_string if l not in string.digits)
    my_str = ' '.join(my_str.split())     # this changes all white spaces to single spaces
    return my_str


def count_chars_words(my_string):
    """Number of characters and words in my_string. Numbers are deemed to be a word."""
    my_str = remove_punctuation(my_string)
    words = my_str.split(' ')
    num_words = len(words)
    num_chars = 0
    for word in words:
        num_chars += len(word)
    return num_chars, num_words


def count_words(my_string):
    """Number of words in my_string. Numbers are deemed to be a word."""
    my_str = remove_punctuation(my_string)
    words = my_str.split(' ')
    num_words = len(words)
    return num_words


def count_sentences(my_string):
    """Count the number of sentences in my_string"""
    my_str = my_string.rstrip()
    sent_ends = '.!?'
    num_sent = 0
    if len(my_str) < 2:
        return num_sent
    else:
        for i in range(2, len(my_str) - 2):
            # check for abbreviations like U.S.A. or e.g. or i.e. followed by a space then upper case letter
            if not(my_str[i] == my_str[i-2] == '.' and my_str[i-1] in string.ascii_letters):
                # check for sentence ending punctuation followed by a space and then an upper case letter
                if my_str[i] in sent_ends and my_str[i + 1] == ' ' and my_str[i + 2] in string.ascii_uppercase:
                    num_sent += 1
    # check special case of last character being a sentence end
    if my_str[-1] in sent_ends:
        num_sent += 1
    return num_sent


def convert_string_to_words(my_string):
    """Convert a string to a list of words"""
    my_str = remove_punctuation(my_string)
    my_str = remove_digits(my_str)
    words = my_str.split(' ')
    return words


def unique_words(my_string):
    """Create a list of unique words that appear in my_string"""
    words = set(convert_string_to_words(my_string))
    return words

def count_in_string(my_string):
    """Count the frequency of words in my_string"""
    words = convert_string_to_words(my_string)
    count = {word: words.count(word) for word in words}
    return count

# STRING FUNCTIONS THAT RELATE TO A DICTIONARY


def sum_in_words(my_string, words):
    """Total count of words in my_string that appear in the list words"""
    my_str = convert_string_to_words(my_string)
    count = sum([1 for word in my_str if word in words])
    return count


def count_in_words(my_string, words):
    """Frequency/count of each word in my_string that appear in the list words"""
    my_str = convert_string_to_words(my_string)
    count = {word: my_str.count(word) for word in words}
    return count


def remove_duplicates(count_dict):
    """Receives a count_in_words dictionary and returns a list of the unique words, i.e. only appear once."""
    unique_words_in_count = [key for key in count_dict.keys() if count_dict[key] == 1]
    return unique_words_in_count


def not_in_dictionary(my_string, words):
    """Returns a list of words that are in my_string, but not in the list words"""
    not_in = [word for word in unique_words(my_string.lower()) if word not in set(words)]
    return not_in


# COLLECT WORD TYPES FROM CORPUS

def short_form_replace_check(my_string):
    """Return a list of all short-form words or words with apostrophes in my_string"""
    result = []
    my_str = my_string
    for j in range(len(my_str)):
        if my_str[j] == '\'':
            n = j
            while my_str[n] != ' ':
                n += 1
                if n == len(my_str) - 1:
                    break
            m = j
            while my_str[m] != ' ':
                m -= 1
                if m == 0:
                    break
            result.append(my_str[m + 1:n].lower())
    result = sorted(list(set(result)))
    return result


def find_non_utf8_chars(my_string):
    """Returns a dict of all non-utf-8 characters with their byte encoded character and frequency from my_string"""
    my_str = my_string
    result = {}
    for char in my_str:
        if len(str(char.encode('utf-8'))) > 5:
            if char in result:
                result[char][1] += 1
            else:
                result[char] = [char.encode('utf-8'), 0]
    return result

def replace_non_utf8_apostrophes(my_str):
    """Replace most common non-utf-8 apostrophes with utf-8 apostrophes"""
    apostrophes = [str(b'\xe2\x80\x98'), str(b'\xe2\x80\x99')]
    for i in range(len(my_str)):
        if str(my_str[i].encode('utf-8')) in apostrophes:
            my_str = my_str[:i] + "'" + my_str[i+1:]
    return my_str



# CHECK THIS!!!!
def fi_replace(my_str):
    """Replace missing "fi's" in words that need them."""
    my_str = re.sub(r"(\s)(\?)([a-z])", r" fi\3", my_str)
    my_str = re.sub(r"(\s)(\?)(\s)([a-z])", r" fi\4", my_str)
    my_str = re.sub(r"([A-z])(\?)([a-z])", r"\1fi\3", my_str)
    my_str = re.sub(r"([A-z])(\?)(\s)([a-z])", r"\1fi\4", my_str)
    return my_str


def replace_non_utf8_chars(my_str):
    """Replace all non-utf8 characters with the user defined characters in the bytes_replace_dict"""
    byte_dict = get_bytes_replace_dict()
    for i in range(len(my_str)):
        if str(my_str[i].encode('utf-8')) in byte_dict:
            byte_key = str(my_str[i].encode('utf-8'))
            my_str = my_str[:i] + byte_dict[byte_key] + my_str[i+1:]
    my_str = my_str.replace('""""', '"')
    my_str = my_str.replace('\\\'', '\'')
    return my_str


def replace_fi_fl(my_str):
    """Replace 'fi' in any word that includes 'fi', where the word is not in the all_words dictionary,
    but where the word with 'fl' in place of 'fi' is in the all_words dictionary"""
    all_words = get_dictionary('All_words.txt')
    words = my_str.split(' ')
    for i in range(len(words)):
        if 'fi' in words[i]:
            if words[i] not in all_words:
                if words[i].replace('fi', 'fl') in all_words:
                    words[i] = words[i].replace('fi', 'fl')
    my_str = ' '.join(words)
    return my_str


def v1_v2_frequency_in_words(my_string1, my_string2, words):
    """Takes two strings and returns word matched frequency vectors ('a' & 'b') for a given dictionary 'words',
     which is useful for cosine similarity analysis."""
    count1 = count_in_words(my_string1, words)  # returns dict(keys:word in words, values:word frequency)
    count2 = count_in_words(my_string2, words)  # returns dict keys:word in words, values:word frequency)
    a = []
    b = []
    for key in count1.keys():                     # create two word frequency vectors 'a' and 'b'
        a.append(count1[key])
        b.append(count2[key])
    return a, b


def ferret(my_string1, my_string2, num_grams=3):
    """calculate the proportion of matching num_grams between two strings. It is useful for
    determining how correlated two strings are hence their relative informational value"""
    # converts strings to single word lists
    words1 = convert_string_to_words(my_string1.lower())
    words2 = convert_string_to_words(my_string2.lower())
    # aggregate single words into multi-word strings or num_grams, default is 3 or trigrams
    num_grams1 = []
    num_grams2 = []
    for i in range(num_grams-1, len(words1)):
        string1 = ''
        for j in range(num_grams - 1, -1, -1):
            if j != 0:
                string1 += words1[i - j] + ' '
            else:
                string1 += words1[i - j]
        num_grams1.append(string1)
    for i in range(num_grams-1, len(words2)):
        string2 = ''
        for j in range(num_grams - 1, -1, -1):
            if j != 0:
                string2 += words2[i - j] + ' '
            else:
                string2 += words2[i - j]
        num_grams2.append(string2)
    # find unique num_grams from my_strings
    num_grams1 = set(num_grams1)
    num_grams2 = set(num_grams2)
    # find union and intersection of the num_grams lists
    num_grams_total = num_grams1.union(num_grams2)
    num_grams_identical = num_grams1.intersection(num_grams2)
    print(num_grams1)
    print(num_grams2)
    print(num_grams_identical)
    print(num_grams_total)
    # calculate ratio of intersection to union of num_grams between my_strings
    if num_grams_total == 0:
        result = ''
    else:
        result = len(num_grams_identical) / len(num_grams_total)
    return result


def coleman_liau(my_string):
    """Returns the Coleman-Liau readability algorithm score for the string my_string"""
    characters, words = count_chars_words(my_string)
    sentences = count_sentences(my_string)
    result = ''
    if words != 0:
        result = 0.0588 * (100 * characters / words) - 0.296 * (100 * sentences / words) - 15.8
    return result


def automated_readability(my_string):
    """Returns the automated readability index algorithm score for the string my_string"""
    characters, words = count_chars_words(my_string)
    sentences = count_sentences(my_string)
    result = ''
    if words != 0 or sentences != 0:
        result = int(4.71 * (characters / words) + 0.5 * (words / sentences) - 21.43) + 1
    return result

def count_in_words(words):
    """Count the frequency of words in words list"""
    count = {word: words.count(word) for word in words}
    return count

def doc_count_in_word_list(words, word_list):
    """Count the frequency of words in a document from a word list"""
    count = {word: words.count(word) for word in word_list}
    return count



In [ ]:
# Get QA Section
start = find_full_string_index(ts, QA_SECTION_HEADERS)
end = find_disclaimer_index(ts,DISCLAIMER)
agg_names = co_part_names + oth_part_names + ['Unknown Oth Part'] + ['Unknown Co Part']
pts = []
process_qa_section(ts, pts, agg_names, start, end)

In [ ]:

def process_qa_section(ts, pts, names, start_index, end_index):

    # get the Q&A section
    qa_section = get_section(ts, start_index, end_index)

    # create dict for comments by each participant
    section_dict = {key: "" for key in names}
    curr_speaker = 'Operator'

    for line in qa_section:
        if len(line) > 0:
            if line == 'Operator':
                curr_speaker = 'Operator'
            if line[0] == "<":
                if line[0:4] == "<Q>:":
                    section_dict['Unknown Oth Part'] += line[5:] + " "
                    curr_speaker = 'Unknown Oth Part'
                if line[0:4] == "<A>:":
                    section_dict['Unknown Co Part'] += line[5:] + " "
                    curr_speaker = 'Unknown Co Part'
                if line[2:5] == " - ":
                    name_end = line.find(">")
                    name = line[5:name_end]
                    if name in section_dict.keys():
                        section_dict[name] += line[name_end+3:] + " "
                    curr_speaker = name
            else:
                if curr_speaker in section_dict.keys():
                    section_dict[curr_speaker] += line + " "

    # print speaker and their comments
    pts.append("Q&A Section: Speaker and Comments")
    for name in names:
        pts.append(name + ": " + section_dict[name])
    return pts



In [ ]:

def get_dir(path1):
    return os.listdir(path1)

def copy_pdf():
    directories = get_dir(EXTRACT_PATH2)
    directories = [EXTRACT_PATH2 + directory + "/" for directory in directories]
    for directory in directories:
        print(directory)
        file_list = get_files(directory, "pdf")
        for file in file_list:
            print(file)
            shutil.copy2(file, PDF_PATH2)

def get_pdf_names(path1):
    pdf_list = glob.glob(path1 + "*.pdf")
    with open(path1 + "PDF_files.txt", "w") as file:
        for pdf in pdf_list:
            file.write(pdf + "\n")

def get_file_details():

    file_list = get_files(myTXT_PATH, "txt")

    for i in file_list:

        with open(i, "r") as file:
            ts = [line.rstrip('\n') for line in file]

        name = ''
        code = ''
        date = ''
        event = ''

        if len(ts) >= 1 and len(ts[0]) > 14:
            name = ts[0][14:]
        if len(ts) >= 2 and len(ts[0]) > 16:
            code = ts[1][16:19]
        if len(ts) >= 3 and len(ts[0]) > 6:
            date = ts[2][6:]
        if len(ts) >= 4 and len(ts[0]) > 19:
            event = ts[3][19:]

        print(i+",", name+",", code+",", date+",", event)

def find_part_string_index(ts, string_list):
    result = -1
    k = 0
    while True:
        for my_str in string_list:
            if ts[k].strip() == my_str:
                result = k
                break
            # Check for small typo (additional leading or lagging chars) in the header - only one is BHP
            if ts[k].find(my_str) != -1 and len(ts[k]) < len(my_str) + 5:
                result = k
                break
        k += 1
        if k == len(ts):
            break
    return result


def get_processed_parts(ts, start_index, end_index):
    result = [name[re.search("[a-z]", name.lower()).start():] for name in ts[start_index+1:end_index] if len(name) >= 3]
    return result



def process_transcript(transcript, processed_transcript, print_ts="N"):

    # print transcript
    if print_ts == "Y":
        for item in transcript:
            print(item)

    # collect indices
    co_parts_index = find_full_string_index(transcript, CO_PART_HEADERS)
    if co_parts_index == -1:
        co_parts_index = find_part_string_index(transcript, CO_PART_HEADERS)

    oth_parts_index = find_full_string_index(transcript, OTH_PART_HEADERS)
    if oth_parts_index == -1:
        oth_parts_index = find_part_string_index(transcript, OTH_PART_HEADERS)

    md_section_index = find_full_string_index(transcript, MD_SECTION_HEADERS)
    if md_section_index == -1:
        md_section_index = find_part_string_index(transcript, MD_SECTION_HEADERS)

    qa_section_index = find_full_string_index(transcript, QA_SECTION_HEADERS)
    if qa_section_index == -1:
        qa_section_index = find_part_string_index(transcript, QA_SECTION_HEADERS)

    disclaimer_index = find_disclaimer_index(transcript, DISCLAIMER)

    problem_inds = [md_section_index, disclaimer_index]

    if -1 in problem_inds:
        processed_transcript.append("Index problem")
        print(problem_inds)
    else:
        # Any company participants found?
        if co_parts_index == -1:
            co_parts_index = md_section_index

        # No other participants found
        if oth_parts_index == -1:
            oth_parts_index = md_section_index

        # Collect and write company participants
        if co_parts_index != md_section_index:
            co_parts = get_parts(transcript, co_parts_index, oth_parts_index)
        else:
            co_parts = ['Unknown Co Part']
        processed_transcript.append("Company Participants")
        for item in co_parts:
            processed_transcript.append(item)
        processed_transcript.append("")

        # Collect and write other participants
        if oth_parts_index != md_section_index:
            oth_parts = get_parts(transcript, oth_parts_index, md_section_index)
        else:
            oth_parts = ['Unknown Oth Part']
        processed_transcript.append("Other Participants")
        for item in oth_parts:
            processed_transcript.append(item)
        processed_transcript.append("")

        # Further checks to find the start of Q&A section
        qa_type = 0
        if qa_section_index == -1:
            k = len(transcript)+1
            for line in range(md_section_index, len(transcript)):
                if transcript[line][0:2] == '<Q':
                    if line < k:
                        k = line
                        qa_type = 1
            if qa_type == 1:
                qa_section_index = k-1
            else:
                if oth_parts != ['Unknown Oth Part']:
                    k = len(transcript)+1
                    for line in range(md_section_index, len(transcript)):
                        for name in oth_parts:
                            if transcript[line].find(name) != -1:
                                if line < k:
                                    k = line
                    if k != len(transcript)+1:
                        qa_section_index = k-1

        # There appears to be no QA section
        if qa_section_index == -1:
            qa_section_index = disclaimer_index

        result = [co_parts_index, oth_parts_index, md_section_index, qa_section_index, disclaimer_index]
        print(result)

        # collect and write comments by participant in the MD section
        if co_parts == ['Unknown Co Part']:
            processed_transcript = process_md_no_names(transcript, processed_transcript, co_parts, md_section_index, qa_section_index)
        else:
            processed_transcript = process_section(transcript, processed_transcript, co_parts, md_section_index, qa_section_index, "MD")
        processed_transcript.append("")

        # collect and write comments by participant in the QA section
        if qa_section_index == disclaimer_index:
            processed_transcript.append("Q&A Section: Speaker and Comments")
            processed_transcript.append("No Q&A section")
        else:
            # collect and print comments by participant in the Q&A section
            all_parts = co_parts + oth_parts + ["Operator"]
            if "Unknown Co Part" not in all_parts:
                all_parts.append("Unknown Co Part")
            if "Unknown Oth Part" not in all_parts:
                all_parts.append("Unknown Oth Part")

            if "Q&A" in transcript:
                processed_transcript = process_qa_section(transcript, processed_transcript, all_parts, qa_section_index, disclaimer_index)
            elif "QUESTION AND ANSWER SECTION"  in transcript:
                processed_transcript = process_qa_section(transcript, processed_transcript, all_parts, qa_section_index, disclaimer_index)
            elif "QUESTION AND ANSWER SESSION" in transcript:
                processed_transcript = process_qa_section(transcript, processed_transcript, all_parts, qa_section_index, disclaimer_index)
            elif qa_type == 1:
                processed_transcript = process_qa_section(transcript, processed_transcript, all_parts, qa_section_index, disclaimer_index)
            else:
                processed_transcript = process_section(transcript, processed_transcript, all_parts, qa_section_index, disclaimer_index, "Q&A")
            processed_transcript.append("")
    return processed_transcript





def process_qa_section(ts, pts, names, start_index, end_index,):

    # get the Q&A section
    qa_section = get_section(ts, start_index, end_index)

    # create dict for comments by each participant
    section_dict = {key: "" for key in names}
    curr_speaker = 'Operator'

    for line in qa_section:
        if len(line) > 0:
            if line == 'Operator':
                curr_speaker = 'Operator'
            if line[0] == "<":
                if line[0:4] == "<Q>:":
                    section_dict['Unknown Oth Part'] += line[5:] + " "
                    curr_speaker = 'Unknown Oth Part'
                if line[0:4] == "<A>:":
                    section_dict['Unknown Co Part'] += line[5:] + " "
                    curr_speaker = 'Unknown Co Part'
                if line[2:5] == " - ":
                    name_end = line.find(">")
                    name = line[5:name_end]
                    if name in section_dict.keys():
                        section_dict[name] += line[name_end+3:] + " "
                    curr_speaker = name
            else:
                if curr_speaker in section_dict.keys():
                    section_dict[curr_speaker] += line + " "

    # print speaker and their comments
    pts.append("Q&A Section: Speaker and Comments")
    for name in names:
        pts.append(name + ": " + section_dict[name])
    return pts


def process_raw_files():
    my_file_list = sorted(get_files(TXT_PATH, "txt"))
    for i, file_path in enumerate(my_file_list):
        print(i, file_path[len(TXT_PATH):])
        pro_ts = [file_path[len(TXT_PATH):]]
        # read in the transcript
        my_text = read_transcript(file_path)
        # check if the file is an earnings call
        if my_text[0].find("Earnings Call") != -1:
            # process the transcript
            pro_ts = process_transcript(my_text, pro_ts, "N")

            process_error = -1
            for item in pro_ts:
                if item.find('Could not find the start ') != -1:
                    process_error = 1
                if item.find('Index problem') != -1:
                    process_error = 1

            if process_error == 1:
                file = open(F_TXT_PATH+pro_ts[0], "w")
            else:
                file = open(P_TXT_PATH+pro_ts[0], "w")
            for item in pro_ts:
                # note the split removes unnecessary white spaces including page breaks
                file.write(' '.join(item.split()) + "\n")
            file.close()


def get_MD(ts, start, end):
    section = ts[start+1:end-1]
    MD_section = ""
    for string in section:
        colon_index = string.find(":")
        if colon_index != len(string)-1:
            MD_section += string[colon_index+2:] + " "
    return MD_section


def get_QA(ts, co_parts, oth_parts, start):
    section = ts[start+1:]
    QA_section_co = ""
    QA_section_oth = ""
    for string in section:
        colon_index = string.find(":")
        part = string[:colon_index]
        if colon_index != len(string)-1:
            if part in co_parts:
                QA_section_co += string[colon_index+2:] + " "
            if part in oth_parts:
                QA_section_oth += string[colon_index+2:] + " "
    return QA_section_co, QA_section_oth


def process_processed_transcript(transcript, dict_list1, dict_list2):

    results = []
    results2 = []

    co_parts_index = find_full_string_index(transcript, ['Company Participants'])
    oth_parts_index = find_full_string_index(transcript, ['Other Participants'])
    md_section_index = find_full_string_index(transcript, ['MD Section: Speaker and Comments'])
    qa_section_index = find_full_string_index(transcript, ['Q&A Section: Speaker and Comments'])

    # collect and print company participants
    co_parts = get_processed_parts(transcript, co_parts_index, oth_parts_index)
    if 'Unknown Co Part' not in co_parts:
        co_parts.append('Unknown Co Part')

    # collect and print other participants
    oth_parts = get_processed_parts(transcript, oth_parts_index, md_section_index)
    if 'Unknown Oth Part' not in co_parts:
        co_parts.append('Unknown Oth Part')

    # collect comments in the MD section
    MD_section = get_MD(transcript, md_section_index, qa_section_index)

    # collect and print comments by participant group in the Q&A section
    no_QA = False
    if 'No Q&A section' in transcript:
        no_QA = True
    else:
        QA_section_co, QA_section_oth = get_QA(transcript, co_parts, oth_parts, qa_section_index)

    # MD_words = tt.convert_string_to_words(tt.clean_text(MD_section.lower()))
    # QA_words_co = tt.convert_string_to_words(tt.clean_text(QA_section_co.lower()))
    # QA_words_oth = tt.convert_string_to_words(tt.clean_text(QA_section_oth.lower()))

    MD_words = tt.clean_text(MD_section.lower())
    if not no_QA:
        QA_words_co = tt.clean_text(QA_section_co.lower())
        QA_words_oth = tt.clean_text(QA_section_oth.lower())

    cv = CountVectorizer(ngram_range=(1,2))
    analyze = cv.build_analyzer()

    st_word_counts = {key: 0 for key in dict_list1}
    lt_word_counts = {key: 0 for key in dict_list2}

    # process MD section
    words = analyze(MD_words)
    word_counts_st = {word: words.count(word) for word in dict_list1}
    word_counts_lt = {word: words.count(word) for word in dict_list2}
    tot_num = len(MD_words)
    st_num = sum(word_counts_st.values())
    lt_num = sum(word_counts_lt.values())
    if st_num+lt_num == 0:
        ratio = "NA"
    else:
        ratio = round(100*st_num/(st_num+lt_num),1)
    # print(st_num, lt_num, ratio, tot_num)
    results.append('MD section')
    results.append(str(word_counts_st))
    results.append(str(word_counts_lt))
    results.append(str(st_num) + " " + str(lt_num) + " " + str(ratio) + " " + str(tot_num))
    results2.append(st_num)
    results2.append(lt_num)
    results2.append(tot_num)
    for key in st_word_counts.keys():
        st_word_counts[key] += word_counts_st[key]
    for key in lt_word_counts.keys():
        lt_word_counts[key] += word_counts_lt[key]

    # process QA sections
    if not no_QA:
        words = analyze(QA_words_co)
        word_counts_st = {word: words.count(word) for word in dict_list1}
        word_counts_lt = {word: words.count(word) for word in dict_list2}
        tot_num = len(QA_words_co)
        st_num = sum(word_counts_st.values())
        lt_num = sum(word_counts_lt.values())
        if st_num+lt_num == 0:
            ratio = "NA"
        else:
            ratio = round(100*st_num/(st_num+lt_num),1)
        results.append('QA Co section')
        results.append(str(word_counts_st))
        results.append(str(word_counts_lt))
        results.append(str(st_num) + " " + str(lt_num) + " " + str(ratio) + " " + str(tot_num))
        results2.append(st_num)
        results2.append(lt_num)
        results2.append(tot_num)
        for key in st_word_counts.keys():
            st_word_counts[key] += word_counts_st[key]
        for key in lt_word_counts.keys():
            lt_word_counts[key] += word_counts_lt[key]
    else:
        results.append('QA Co section')
        results.append('No QA section')
        results2.append('NA')
        results2.append('NA')
        results2.append('NA')

    if not no_QA:
        words = analyze(QA_words_oth)
        word_counts_st = {word: words.count(word) for word in dict_list1}
        word_counts_lt = {word: words.count(word) for word in dict_list2}
        tot_num = len(QA_words_oth)
        st_num = sum(word_counts_st.values())
        lt_num = sum(word_counts_lt.values())
        if st_num+lt_num == 0:
            ratio = "NA"
        else:
            ratio = round(100*st_num/(st_num+lt_num),1)
        results.append('QA Oth section')
        results.append(str(word_counts_st))
        results.append(str(word_counts_lt))
        results.append(str(st_num) + " " + str(lt_num) + " " + str(ratio) + " " + str(tot_num))
        results2.append(st_num)
        results2.append(lt_num)
        results2.append(tot_num)
        for key in st_word_counts.keys():
            st_word_counts[key] += word_counts_st[key]
        for key in lt_word_counts.keys():
            lt_word_counts[key] += word_counts_lt[key]
    else:
        results.append('QA Oth section')
        results.append('No QA section')
        results2.append('NA')
        results2.append('NA')
        results2.append('NA')

#    return results
    return results2, st_word_counts, lt_word_counts


def process_processed_files():
    my_file_list = sorted(get_files(P_TXT_PATH, "txt"))
    # empty results file
    file = open(RESULTS_PATH + "results.csv", "w")
    file.write("File,MD_st,MD_lt,MD_tot,QACo_st,QACo_lt,QACo_tot,QAOth_st,QAOth_lt,QAOth_tot" + "\n")
    file.close()
    # Create empty dictionaries
    st_words = {key: 0 for key in SHORT_TERM}
    lt_words = {key: 0 for key in LONG_TERM}
    # crunch numbers for each transcript
    for i, file_path in enumerate(my_file_list):
        print(i, file_path[len(P_TXT_PATH):])
        my_text = read_transcript(file_path)
        result, st_word_counts, lt_word_counts = process_processed_transcript(my_text, SHORT_TERM, LONG_TERM)
        for key in st_words.keys():
            st_words[key] += st_word_counts[key]
        for key in lt_words.keys():
            lt_words[key] += lt_word_counts[key]
        # append to file
        file = open(RESULTS_PATH + "results.csv", "a")
        file.write(file_path[len(P_TXT_PATH):] + ",")
        file.write(str(result[0]))
        for item in result[1:]:
            file.write("," + str(item))
        file.write("\n")
        file.close()
    print("ST words")
    for key, value in st_words.items():
        print(key + ": " + str(value))
    print("LT words")
    for key, value in lt_words.items():
        print(key + ": " + str(value))


def process_processed_file(file_name):
    # Create empty dictionaries
    st_words = {key: 0 for key in SHORT_TERM}
    lt_words = {key: 0 for key in LONG_TERM}
    # crunch numbers for each transcript

    my_text = read_transcript(P_TXT_PATH + file_name)
    result, st_word_counts, lt_word_counts = process_processed_transcript(my_text)
    for key in st_words.keys():
        st_words[key] += st_word_counts[key]
    for key in lt_words.keys():
        lt_words[key] += lt_word_counts[key]
    print(file_name)
    my_str = str(result[0])
    for item in result[1:]:
        my_str += "," + str(item)
    print(my_str)
    print("ST words")
    for key, value in st_words.items():
        print(key + ": " + str(value))
    print("LT words")
    for key, value in lt_words.items():
        print(key + ": " + str(value))


def process_raw_file(file_name):
    # read in the transcript
    my_text = read_transcript(TXT_PATH + file_name)
    # check if the file is an earnings call
    pro_ts = [file_name]
    if my_text[0].find("Earnings Call") != -1:
        # process the transcript
        pro_ts = process_transcript(my_text, pro_ts, "Y")
        process_error = -1
        for item in pro_ts:
            # note the split removes unnecessary white spaces including page breaks
            print(' '.join(item.split()) + "\n")


def alpha_ideas_compare_two_dicts(files_path, results_path, result_file, dict1, dict2):

    # get list of pre-processed file names and path
    my_file_list = sorted(get_files(files_path, "txt"))

    # Ensure the results file is empty and write a header
    with open(results_path + result_file, "w") as my_result_file:
        my_result_file.write("File,MD_d1,MD_d2,MD_tot,QACo_d1,QACo_d2,QACo_tot,QAOth_d1,QAOth_d2,QAOth_tot" + "\n")

    # Load the two dictionaries as lists
    my_dict_list1 = tt.get_dictionary(dict1)
    my_dict_list2 = tt.get_dictionary(dict2)

    # Create empty dictionaries
    d1_words = {key: 0 for key in my_dict_list1}
    d2_words = {key: 0 for key in my_dict_list2}

    # Crunch numbers for each transcript
    for i, file_path in enumerate(my_file_list):

        # print file index and name to monitor progress
        print(i, file_path[len(files_path):])

        # read in the pre-processed transcript
        my_text = read_transcript(file_path)

        # process the transcript
        result, d1_word_counts, d2_word_counts = process_processed_transcript(my_text, my_dict_list1, my_dict_list2)

        for key in d1_words.keys():
            d1_words[key] += d1_word_counts[key]
        for key in d2_words.keys():
            d2_words[key] += d2_word_counts[key]

        # append to file
        with open(results_path + result_file, "a") as my_result_file:
            my_result_file.write(file_path[len(files_path):] + ",")
            my_result_file.write(str(result[0]))
            for item in result[1:]:
                my_result_file.write("," + str(item))
            my_result_file.write("\n")

    # print summary of word count results for the two dictionaries
    print("D1 words")
    for key, value in d1_words.items():
        print(key + ": " + str(value))

    print("D2 words")
    for key, value in d2_words.items():
        print(key + ": " + str(value))

# my_file = "PRU-2012-02---70.txt"
# alpha_ideas_compare_two_dicts(P_TXT_PATH, ALPHA_RESULTS + "Tone/", "Tone.csv", "Positive_CS.txt", "Negative_CS.txt")
# process_raw_files()
# process_processed_files()
# process_raw_file(my_file)
# process_processed_file(my_file)

